In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%matplotlib inline
!pip install -qqq greykite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.7/22.7 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.9/499.9 kB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 65.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 40.1 MB/s et

In [ ]:
# !pip install -qqq pandas
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Measles/nigeria.csv')

df['Year-Month'] = df['Year-Month'].astype('datetime64[ns]')
df.rename(columns = {'Year-Month': 'ts', 'Cases': 'y'}, inplace = True)
df

,ts,y
0,2011-01-01,4539
1,2011-02-01,4025
2,2011-03-01,4533
3,2011-04-01,2520
4,2011-05-01,1057
...,...,...
154,2023-11-01,200
155,2023-12-01,134
156,2024-01-01,2132
157,2024-02-01,696


In [ ]:
from collections import defaultdict
import warnings

warnings.filterwarnings("ignore")

import pandas as pd
import plotly

import pandas as pd
from greykite.framework.templates.forecaster import Forecaster
from greykite.framework.templates.autogen.forecast_config import ForecastConfig
from greykite.framework.templates.autogen.forecast_config import MetadataParam
from greykite.framework.templates.autogen.forecast_config import ModelComponentsParam
from greykite.framework.templates.autogen.forecast_config import EvaluationPeriodParam
from greykite.framework.templates.autogen.forecast_config import EvaluationMetricParam



from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.framework.utils.result_summary import summarize_grid_search_results

In [ ]:
# Specifies dataset information
metadata = MetadataParam(
     time_col="ts",  # name of the time column
     value_col="y",  # name of the value column
     freq="MS"  #"MS" for Montly at start date, "H" for hourly, "D" for daily, "W" for weekly, etc.
 )

model_components = ModelComponentsParam(
    seasonality={
        "yearly_seasonality":12    #12

    },
    growth={
        "growth_term": "quadratic"   #linear, quadratic, sqrt, None
    },
    events={
        "holidays_to_model_separately": None  # "auto",["New Year's Day", "Thanksgiving"],None]
    },
     changepoints={
        "changepoints_dict": {
            "method": "auto",  # 'auto' to automatically detect changepoints
            "regularization_strength": 0.5,  # Regularization strength to control number of changepoints
            "potential_changepoint_n": 11,  # Number of potential changepoints to consider
            "no_changepoint_proportion_from_end": 0.1  # Proportion of data at the end where no changepoint is allowed
        }
    },

    uncertainty={
        "uncertainty_dict":  "auto"  #`dict` or "auto" or None

    },
    custom={
        "fit_algorithm_dict": {
            "fit_algorithm": "elastic_net"    #ridge, lasso, elastic_net, linear, quantile_regression
        }
    }

)

In [ ]:
# Specifies dataset information
metadata = MetadataParam(
     time_col="ts",  # name of the time column
     value_col="y",  # name of the value column
     freq="MS"  #"MS" for Montly at start date, "H" for hourly, "D" for daily, "W" for weekly, etc.
 )

forecaster = Forecaster()
result = forecaster.run_forecast_config(
     df=df,
     config=ForecastConfig(
         model_template=ModelTemplateEnum.SILVERKITE.name,
         forecast_horizon=12,
         coverage=0.95,  # 95% prediction intervals
         metadata_param=metadata,
         model_components_param=model_components

    )
)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


In [ ]:
ts = result.timeseries
fig = ts.plot()
plotly.io.show(fig)

In [ ]:
backtest = result.backtest
fig = backtest.plot()
plotly.io.show(fig)

In [ ]:
forecast_testt = result.backtest.df_test.forecast
actual_testt = result.backtest.df_test.actual

In [ ]:
forecast_testt

147    1573.456553
148    1371.061975
149     909.596117
150    1057.680104
151     817.663969
152     823.649992
153     725.725927
154     614.222031
155     635.529756
156    1584.716104
157    2007.817437
158    2291.921912
Name: forecast, dtype: float64

In [ ]:
actual_testt

147    1747
148    1598
149    1113
150     613
151     321
152     196
153     245
154     200
155     134
156    2132
157     696
158      12
Name: actual, dtype: int64

In [ ]:
from sklearn.metrics import mean_squared_error
error = np.sqrt(mean_squared_error(actual_testt,forecast_testt))
#print('Test RMSE: %.4f' % error)

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(actual_testt,forecast_testt)
#print("Mean Absolute Error (MAE):", mae)

from sklearn.metrics import mean_squared_log_error
msle = mean_squared_log_error(actual_testt,forecast_testt)
#print("Mean Squared Logarithmic Error (MSLE):", msle)

from sklearn.metrics import mean_absolute_percentage_error
#print('Mean Absolute Percentage Error (MAPE):',mean_absolute_percentage_error(valid["Cases"],predictions))

print('RMSE: %.2f' % error)
def mean_absolute_percentage_error(y_true, y_pred):

    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


mape = mean_absolute_percentage_error(actual_testt,forecast_testt)
print(f"MAPE: {mape:.2f}%")
print('MAE:  %.2f' % mae)
print('MSLE: %.2f' % msle)

RMSE: 858.00
MAPE: 1715.08%
MAE:  642.37
MSLE: 3.01


In [ ]:
backtest_eval = defaultdict(list)
for metric, value in backtest.train_evaluation.items():
    backtest_eval[metric].append(value)
    backtest_eval[metric].append(backtest.test_evaluation[metric])
metrics = pd.DataFrame(backtest_eval, index=["train", "test"]).T
metrics

,train,test
CORR,0.574305,0.313792
R2,0.236487,-0.536595
MSE,3884621.367697,736165.776761
RMSE,1970.944283,858.001035
MAE,1044.352953,642.365032
MedAE,640.301478,488.694948
MAPE,219.895316,1715.083835
MedAPE,79.551714,171.601747
sMAPE,39.007533,40.207203
Q80,522.176476,186.031469


In [ ]:
# Access forecasted values
forecast_values = result.forecast.df

# Access forecasted components (e.g., trend, seasonality)
forecast_components = result.backtest.df_test
forecast_asd = result.forecast.df_test

# Print or display these results as needed
print("Forecast Values:")
print(forecast_components)


Forecast Values:
            ts  actual     forecast  forecast_lower  forecast_upper
147 2023-04-01    1747  1573.456553    -2302.730065     5449.643171
148 2023-05-01    1598  1371.061975    -2505.124642     5247.248593
149 2023-06-01    1113   909.596117    -2966.590501     4785.782735
150 2023-07-01     613  1057.680104    -2818.506514     4933.866722
151 2023-08-01     321   817.663969    -3058.522649     4693.850587
152 2023-09-01     196   823.649992    -3052.536626     4699.836609
153 2023-10-01     245   725.725927    -3150.460691     4601.912545
154 2023-11-01     200   614.222031    -3261.964586     4490.408649
155 2023-12-01     134   635.529756    -3240.656862     4511.716374
156 2024-01-01    2132  1584.716104    -2291.470514     5460.902721
157 2024-02-01     696  2007.817437    -1868.369181     5884.004055
158 2024-03-01      12  2291.921912    -1584.264706     6168.108530
